### Analyzing the Effects of Top/Bottom Coding on The Accuracy of Exponential Smoothing Forecasts

***

***

## Import Modules

Note the functions imported from `helper_functions`, as these are custom functions written by the paper authors. See `helper_functions.py` for comments and functions descriptions.

In [1]:
# general modules
import pandas as pd
import numpy as np
import sktime

# import exponential smoothing forecasting model
from sktime.forecasting.exp_smoothing import ExponentialSmoothing

# functions for transformation+forecasting pipeline
from sktime.forecasting.compose import TransformedTargetForecaster

# time series transformations
from sktime.transformations.series.detrend import ConditionalDeseasonalizer, Detrender

##### the `helper_functions.py` file contains many custom functions we wrote to aid in our analysis
##### `full_coding_analysis` combines all of the following - train-test split data,
##### data protection, train models, compare accuracies, return accuracy results
from helper_functions import full_coding_analysis

# suppress warnings from exponential smoothing model not converging
import warnings
warnings.filterwarnings('ignore')

Import data.

In [2]:
# import weekly finance time series
Y = np.genfromtxt("../../Data/Train/Clean/weekly_finance_clean.csv", delimiter = ',', skip_header = 1)
Y = pd.DataFrame(Y)

In [3]:
# detrender = Detrender()
# detrended_series = [detrender.fit_transform(series) for _ , series in Y.iterrows()]
# detrended_series = [i+np.abs(np.min(i))+1.0 for i in detrended_series]
# Y = pd.concat(detrended_series, axis=1).T

## SES

In [4]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        #("deseasonalize", ConditionalDeseasonalizer(model="additive", sp=52)),
        ("forecast", ExponentialSmoothing(use_boxcox=False)),
    ]
)

In [5]:
results_dict_ses = {}
fcasts_ses = {}
fcasts_protected_ses = {}
tests = {}
types = ["Top", "Bottom"]
percentages = [0.10, 0.20, 0.40]
horizons = [1, 20]

In [6]:
for t in types:
    for p in percentages:
        for h in horizons:
            idx = "h="+str(h)+", "+t+" "+str(p)
            results_dict_ses[idx], tests[idx], fcasts_ses[idx], fcasts_protected_ses[idx] = full_coding_analysis(time_series_data=Y,
                                                                                                                 forecasting_model=forecaster,
                                                                                                                 forecast_horizon=h, 
                                                                                                                 coding_type=t, 
                                                                                                                 coding_percentage=p)

In [7]:
results_dict_ses

{'h=1, Top 0.1': {'Mean Accuracies': array([2.18, 2.18]),
  'Protected Mean Accuracies:': array([3.98, 3.98]),
  '% Change Mean accuracy:': array([-82.96, -82.96]),
  '% Change Median accuracy:': array([-192.85, -192.85]),
  '% Forecasted Points adjusted downward:': 49.39,
  '% Forecasted Points adjusted upward:': 50.61,
  '% Series with improved accuracy:': array([28.05, 28.05]),
  '% Series with reduced accuracy:': array([71.95, 71.95]),
  'Original Mean Absolute Error Upward Adjusted:': 2.98,
  'Original Mean Absolute Error Downward Adjusted:': 1.35,
  'Protected Mean Absolute Error Upward Adjusted:': 3.06,
  'Protected Mean Absolute Error Downward Adjusted:': 4.93},
 'h=20, Top 0.1': {'Mean Accuracies': array([4.45, 5.43]),
  'Protected Mean Accuracies:': array([5.86, 6.81]),
  '% Change Mean accuracy:': array([-31.63, -25.36]),
  '% Change Median accuracy:': array([-46.35, -38.39]),
  '% Forecasted Points adjusted downward:': 50.0,
  '% Forecasted Points adjusted upward:': 50.0,
 

In [8]:
original_forecasts = fcasts_ses['h=20, Bottom 0.1']
protected_forecasts = fcasts_protected_ses['h=20, Bottom 0.1']
test = tests['h=20, Bottom 0.1']

In [9]:
adjusted_up = original_forecasts < protected_forecasts
adjusted_up = pd.concat([row for i, row in adjusted_up.iterrows()])
adjusted_down = original_forecasts > protected_forecasts
adjusted_down = pd.concat([row for i, row in adjusted_down.iterrows()])

In [10]:
absolute_error_original = np.absolute(test - original_forecasts)
absolute_error_protected = np.absolute(test - protected_forecasts)

In [11]:
improved = absolute_error_original > absolute_error_protected
improved = pd.concat([row for i, row in improved.iterrows()])
worsened = absolute_error_original < absolute_error_protected
worsened = pd.concat([row for i, row in worsened.iterrows()])

In [12]:
np.mean(adjusted_down[improved])

0.9084645669291339

In [13]:
np.mean(adjusted_up[improved])

0.09153543307086615

***
***

## DES

In [14]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        #("deseasonalize", ConditionalDeseasonalizer(model="additive", sp=52)),
        ("forecast", ExponentialSmoothing(trend="additive", use_boxcox=False)),
    ]
)

In [15]:
results_dict_des = {}
fcasts_des = {}
fcasts_protected_des = {}
tests = {}
types = ["Top", "Bottom"]
percentages = [0.10, 0.20, 0.40]
horizons = [1, 20]

In [16]:
for t in types:
    for p in percentages:
        for h in horizons:
            idx = "h="+str(h)+", "+t+" "+str(p)
            results_dict_des[idx], tests[idx], fcasts_des[idx], fcasts_protected_des[idx] = full_coding_analysis(time_series_data=Y,
                                                                                                                 forecasting_model=forecaster,
                                                                                                                 forecast_horizon=h, 
                                                                                                                 coding_type=t, 
                                                                                                                 coding_percentage=p)

In [17]:
results_dict_des

{'h=1, Top 0.1': {'Mean Accuracies': array([2.1, 2.1]),
  'Protected Mean Accuracies:': array([3.81, 3.81]),
  '% Change Mean accuracy:': array([-81.56, -81.56]),
  '% Change Median accuracy:': array([-160.28, -160.28]),
  '% Forecasted Points adjusted downward:': 50.0,
  '% Forecasted Points adjusted upward:': 50.0,
  '% Series with improved accuracy:': array([29.88, 29.88]),
  '% Series with reduced accuracy:': array([70.12, 70.12]),
  'Original Mean Absolute Error Upward Adjusted:': 2.88,
  'Original Mean Absolute Error Downward Adjusted:': 1.31,
  'Protected Mean Absolute Error Upward Adjusted:': 2.9499999999999997,
  'Protected Mean Absolute Error Downward Adjusted:': 4.66},
 'h=20, Top 0.1': {'Mean Accuracies': array([4.28, 5.24]),
  'Protected Mean Accuracies:': array([5.17, 6.09]),
  '% Change Mean accuracy:': array([-20.72, -16.21]),
  '% Change Median accuracy:': array([-27.45, -20.34]),
  '% Forecasted Points adjusted downward:': 53.959999999999994,
  '% Forecasted Points ad

***
***

## TES

In [18]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(trend="additive",
                                          seasonal="additive",
                                          sp=52,
                                          damped_trend=False, 
                                          use_boxcox=False)),
    ]
)

In [19]:
results_dict_tes = {}
fcasts_tes = {}
fcasts_protected_tes = {}
tests = {}
types = ["Top", "Bottom"]
percentages = [0.10, 0.20, 0.40]
horizons = [1, 20]

In [20]:
for t in types:
    for p in percentages:
        for h in horizons:
            idx = "h="+str(h)+", "+t+" "+str(p)
            results_dict_tes[idx], tests[idx], fcasts_tes[idx], fcasts_protected_tes[idx] = full_coding_analysis(time_series_data=Y,
                                                                                                                 forecasting_model=forecaster,
                                                                                                                 forecast_horizon=h, 
                                                                                                                 coding_type=t, 
                                                                                                                 coding_percentage=p)

In [21]:
results_dict_tes

{'h=1, Top 0.1': {'Mean Accuracies': array([2.44, 2.44]),
  'Protected Mean Accuracies:': array([4.18, 4.18]),
  '% Change Mean accuracy:': array([-71.22, -71.22]),
  '% Change Median accuracy:': array([-125.25, -125.25]),
  '% Forecasted Points adjusted downward:': 56.71000000000001,
  '% Forecasted Points adjusted upward:': 43.29,
  '% Series with improved accuracy:': array([33.54, 33.54]),
  '% Series with reduced accuracy:': array([66.46, 66.46]),
  'Original Mean Absolute Error Upward Adjusted:': 3.2,
  'Original Mean Absolute Error Downward Adjusted:': 1.8599999999999999,
  'Protected Mean Absolute Error Upward Adjusted:': 3.0700000000000003,
  'Protected Mean Absolute Error Downward Adjusted:': 5.029999999999999},
 'h=20, Top 0.1': {'Mean Accuracies': array([4.75, 5.74]),
  'Protected Mean Accuracies:': array([5.65, 6.56]),
  '% Change Mean accuracy:': array([-19.05, -14.28]),
  '% Change Median accuracy:': array([-29.55, -14.26]),
  '% Forecasted Points adjusted downward:': 62.